<a href="https://colab.research.google.com/github/DaeSeokSong/summarize-study/blob/master/%5BPractice%5D%20Computer%20Vision%20Model/RCNN/Faster_RCNN_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab_Notebooks/FasterRCNN/Faster-RCNN-with-torchvision-master/

/content/drive/MyDrive/Colab_Notebooks/FasterRCNN/Faster-RCNN-with-torchvision-master


In [ ]:
import utils
import dataset.transforms as T
import datetime
import os
import time

import torch
import torch.utils.data
from torch import nn
from dataset.coco_utils import get_coco, get_coco_kp
from engine import train_one_epoch, evaluate
from dataset.group_by_aspect_ratio import GroupedBatchSampler, create_aspect_ratio_groups
import argparse
import torchvision

import cv2
import random


In [ ]:

data_path = '../../data/coco/'
model_name = 'fasterrcnn_resnet50_fpn'
dataset_name = 'coco'
device_type = 'cuda'
batch_size = 8
epochs = 20
workers = 8
lr = 0.02
momentum = 0.9 
weight_decay = 0.0001
print_freq = 20
lr_step_size = 8
lr_steps = [8,11]
lr_gamma = 0.1
resume = ''
test_only = True
output_dir = './result'
aspect_ratio_group_factor = 0
pretrained = True
distributed = False
parallel = False
world_size =1
dist_url = 'env://'



In [ ]:
def get_dataset(name, image_set, transform):
    paths = {
        "coco": ('../../data/coco/', get_coco, 91),
        "coco_kp": ('/datasets01/COCO/022719/', get_coco_kp, 2)
    }
    p, ds_fn, num_classes = paths[name]

    ds = ds_fn(p, image_set=image_set, transforms=transform)
    return ds, num_classes

In [ ]:

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
def main():
    if output_dir:
        utils.mkdir(output_dir)    

    # Data loading
    print("Loading data")
    dataset, num_classes = get_dataset(dataset_name, "train", get_transform(train=True))
    dataset_test, _ = get_dataset(dataset_name, "val", get_transform(train=False))    

    print("Creating data loaders")
    if distributed:
        train_sampler = torch.utils.data.distributed.DistributedSampler(dataset)
        test_sampler = torch.utils.data.distributed.DistributedSampler(dataset_test)
    else:
        train_sampler = torch.utils.data.RandomSampler(dataset)
        test_sampler = torch.utils.data.SequentialSampler(dataset_test)

    if aspect_ratio_group_factor >= 0:
        group_ids = create_aspect_ratio_groups(dataset, k=aspect_ratio_group_factor)
        train_batch_sampler = GroupedBatchSampler(train_sampler, group_ids, batch_size)
    else:
        train_batch_sampler = torch.utils.data.BatchSampler(
            train_sampler, batch_size, drop_last=True)

    data_loader = torch.utils.data.DataLoader(
        dataset, batch_sampler=train_batch_sampler, num_workers=workers,
        collate_fn=utils.collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=batch_size,
        sampler=test_sampler, num_workers=workers,
        collate_fn=utils.collate_fn)

    # Model creating
    print("Creating model")
    # model = models.__dict__[model](num_classes=num_classes, pretrained=pretrained)   
    model = torchvision.models.detection.__dict__[model_name](num_classes=num_classes,
                                                              pretrained=pretrained)

    device = torch.device(device_type)
    model.to(device)

    # Distribute
    model_without_ddp = model
    if distributed:
        model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[gpu])
        model_without_ddp = model.module    

    # Parallel
    if parallel:
        print('Training parallel')
        model = torch.nn.DataParallel(model).cuda()
        model_without_ddp = model.module

    # Optimizer
    params = [p for p in model.parameters() if p.requires_grad]

    optimizer = torch.optim.SGD(
        params, lr=lr, momentum=momentum, weight_decay=weight_decay)

    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=lr_steps, gamma=lr_gamma)

    # Resume training
    if resume:
        print('Resume training')
        checkpoint = torch.load(resume, map_location='cpu')
        model_without_ddp.load_state_dict(checkpoint['model'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])

    if test_only:
        evaluate(model, data_loader_test, device=device)
        return

    # Training
    print('Start training')
    start_time = time.time()
    for epoch in range(epochs):
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq)
        lr_scheduler.step()
        if output_dir:
            utils.save_on_master({
                'model': model_without_ddp.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict()
                },
                os.path.join(output_dir, 'model_{}.pth'.format(epoch)))

        # evaluate after every epoch
        evaluate(model, data_loader_test, device=device)

    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str))

In [ ]:
main()


Loading data
loading annotations into memory...
Done (t=18.48s)
creating index...
index created!
loading annotations into memory...
Done (t=0.52s)
creating index...
index created!
Creating data loaders
Using [0, 1.0, inf] as bins for aspect ratio quantization
Count of instances per bin: [85308 31958]
Creating model


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weigh

Test:  [  0/625]  eta: 0:50:24  model_time: 2.4013 (2.4013)  evaluator_time: 0.0688 (0.0688)  time: 4.8394  data: 2.2993  max mem: 4830
Test:  [100/625]  eta: 0:06:58  model_time: 0.6577 (0.6733)  evaluator_time: 0.0543 (0.0516)  time: 0.7661  data: 0.0339  max mem: 5860
Test:  [200/625]  eta: 0:05:30  model_time: 0.6592 (0.6639)  evaluator_time: 0.0506 (0.0523)  time: 0.7745  data: 0.0348  max mem: 5895
Test:  [300/625]  eta: 0:04:11  model_time: 0.6599 (0.6584)  evaluator_time: 0.0483 (0.0597)  time: 0.7452  data: 0.0332  max mem: 5895
Test:  [400/625]  eta: 0:02:53  model_time: 0.6604 (0.6569)  evaluator_time: 0.0414 (0.0574)  time: 0.7556  data: 0.0364  max mem: 5895
Test:  [500/625]  eta: 0:01:35  model_time: 0.6327 (0.6546)  evaluator_time: 0.0407 (0.0565)  time: 0.7238  data: 0.0334  max mem: 5895
Test:  [600/625]  eta: 0:00:19  model_time: 0.6592 (0.6529)  evaluator_time: 0.0551 (0.0569)  time: 0.7379  data: 0.0366  max mem: 5895
Test:  [624/625]  eta: 0:00:00  model_time: 0.66